In [18]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 453 kB 52.0 MB/s 


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#전공데이터셋을 받아옵니다
data = pd.read_csv('/content/drive/MyDrive/rapa/융합프로젝트/전공데이터.csv', low_memory=False)
data.head(2)

,대학명,단대명,학부/학과명,커리큘럼
0,서울대학교,공과대학,컴퓨터공학부,"컴퓨터과학이 여는 세계,컴퓨터과학적 사고와 실습,인공지능 입문,\t인터넷 보안과 프..."
1,서울대학교,미술대학,동양화과,"한국미술과 문화\t,서양미술사,동양화 기초,\t평면조형,평면조형,입체조형,\t기초시..."


In [25]:
from konlpy.tag import Okt
okt = Okt()
# =======================================================    
# 형태소(POS)가 명사,동사,알파벳,숫자에 해당되는 단어 추출
# 정규화(normalization) 어간추출(stemming) 처리
# =======================================================    
def tokenizer(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return(o)

tokenizer( data['커리큘럼'][0])

['컴퓨터과학',
 '세계',
 '컴퓨터과학',
 '사고',
 '실습',
 '인공',
 '지능',
 '입문',
 '인터넷',
 '보안',
 '프라이버시',
 '이산수학',
 '프로그래밍',
 '연습',
 '공학',
 '수학',
 '전기',
 '전자회로',
 '컴퓨터',
 '구조',
 '컴퓨터',
 '프로그래밍',
 '논리',
 '설계',
 '자료구조',
 '컴퓨터공학',
 '세미나',
 '프로그래밍',
 '원리',
 '알고리즘',
 '시스템',
 '프로그래밍',
 '오토마타',
 '이론',
 '운영체제',
 '하드웨어',
 '시스템',
 '설계',
 '프로그래밍언어',
 '선형',
 '비선형',
 '계산',
 '모델',
 '디지털',
 '신호처리',
 '창의',
 '통합',
 '설계',
 '데이터마이닝',
 '개론',
 '데이터베이스',
 '데이터',
 '통신',
 '소프트웨어',
 '개발',
 '원리',
 '실습',
 'IT',
 '창업',
 '개론',
 '임베디드',
 '시스템',
 '응용',
 '소프트웨어',
 '공학',
 '소프트웨어',
 '응용',
 '모바일',
 '컴퓨팅',
 '응용',
 '인공',
 '지능',
 '컴파일러',
 '컴퓨터그래픽스',
 '컴퓨터',
 '네트워크',
 '컴퓨터',
 '모델링',
 '멀티코어',
 '컴퓨팅',
 '컴퓨터',
 '보안',
 '정보',
 '시스템',
 'IT',
 '리더십',
 '세미나',
 '컴퓨터',
 '융합',
 '응용',
 '인간',
 '컴퓨터',
 '상호작용',
 '소셜',
 '네트워크',
 '분석',
 '기계학습',
 '개론',
 '창의',
 '통합',
 '설계',
 '컴퓨터',
 '비다',
 '컴퓨터',
 '기술',
 '특강',
 '인터넷',
 '보안',
 '양자',
 '컴퓨팅',
 '정보',
 '기초',
 '블록',
 '체인',
 '이해',
 '러닝',
 '기초']

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorize = TfidfVectorizer(
    tokenizer=tokenizer, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

tfidf_matrix = tfidf_vectorize.fit_transform(data['커리큘럼'])

In [31]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (51, 51)


In [32]:
#기존 데이터프레임으로부터 전공의 이름을 key, 전공의 인덱스를 value로 하는 딕셔너리 title_to_index를 만들어둡니다.
title_to_index = dict(zip(data['학부/학과명'], data.index))


In [33]:
# 전공명 의 인덱스를 리턴
idx = title_to_index['컴퓨터공학부']
print(idx)

0


In [23]:
X#선택한 전공명을 입력하면 코사인 유사도를 통해 가장 커리큘럼이 유사한 10개의 학과를 찾아내는 함수를 만듭니다.
def get_recommendations(major, cosine_sim=cosine_sim):
    # 선택한 전공명으로부터 해당 전공의 인덱스를 받아온다.
    idx = title_to_index[major]

    # 해당 전공과 모든 전공과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 전공들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 전공들을 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 전공의 인덱스를 얻는다.
    major_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 전공의 이름을 리턴한다.
    return data['학부/학과명'].iloc[major_indices]

In [34]:
#전공 시각디자인학전공과 유사한 전공들을 찾아보겠습니다.

get_recommendations('시각디자인학전공')

5       디자인학부(디자인)
20         산업정보디자인
47     커뮤니케이션디자인학과
21            조형미술
31    디자인학과(시각디자인)
7            정보문화학
32    디자인학과(써피스디자인
11        산업디자인학전공
13        디지털아트학전공
44       컬처테크놀로지전공
Name: 학부/학과명, dtype: object

In [35]:
#전공 미래모빌리티학과와 유사한 전공들을 찾아보겠습니다.

get_recommendations('미래모빌리티학과')

15                                  컴퓨터학
24                          컴퓨터융합소프트웨어학과
14                               소프트웨어전공
35                               소프트웨어학과
10                                 인공지능학
48                            신산업소프트웨어전공
33                   글로벌융합학부 데이터사이언스융합전공
0                                 컴퓨터공학부
50                              산업인공지능전공
29    인공지능(Artificial Intelligence) 연계전공
Name: 학부/학과명, dtype: object